In [7]:
from pathlib import Path
import os
import sys
import json
import torch
import numpy as np

In [2]:
base_dir = Path('/mnt/data/eth3d_processed_monosdf')

In [25]:
for scene in sorted(base_dir.iterdir()):
    transform_path = scene / 'transforms.json'
    with open(transform_path, 'r') as f:
        transforms = json.load(f)
    frames = transforms['frames']

    if all([(key in transforms) for key in ['fl_x', 'fl_y', 'cx', 'cy']]):
        K = np.eye(4)
        K[0, 0] = transforms['fl_x']
        K[1, 1] = transforms['fl_y']
        K[0, 2] = transforms['cx']
        K[1, 2] = transforms['cy']
        intrinsic = K
        focal_length = K[0, 0]

    file_num = len(frames)
    cameras = {}
    for index in range(file_num):
        frame = frames[index]
        pose = np.array(frame['transform_matrix']).reshape(4, 4)
        pose[:, 1:3] *= -1
        cameras["scale_mat_%d"%(index)] = np.eye(4)
        cameras["world_mat_%d"%(index)] = K @ np.linalg.inv(pose)

    out_path = str(scene)
    np.savez(os.path.join(out_path, "cameras.npz"), **cameras)


In [24]:
K

array([[529.7778922 ,   0.        , 481.61879324,   0.        ],
       [  0.        , 529.20144683, 318.17583796,   0.        ],
       [  0.        ,   0.        ,   1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [18]:
np.array(frame['transform_matrix']).view(4,4)

ValueError: Type must be a sub-type of ndarray type